<a href="https://colab.research.google.com/github/suwarnalatha-m/Task-1-Big-Data-Analysis/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install PySpark
!pip install pyspark

In [ ]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("CODTECH Big Data Analysis") \
    .getOrCreate()

print("Spark Session Created Successfully")

Spark Session Created Successfully


In [ ]:
# To check the format of the files
!ls /content

 online_retail.csv  'Online Retail.xlsx'   sample_data


In [ ]:
# CONVERTING Excel → CSV
df_excel = pd.read_excel("/content/Online Retail.xlsx")
df_excel.to_csv("/content/online_retail.csv", index=False)

print("Conversion Completed")

Conversion Completed


In [ ]:
# To check the file is in csv format
!ls /content

 online_retail.csv  'Online Retail.xlsx'   sample_data


In [ ]:
# Loading Dataset using Pyspark
df = spark.read.csv(
    "/content/online_retail.csv",
    header=True,
    inferSchema=True
)

In [40]:
# Preview
display(
    df.limit(10).toPandas().style
    .set_caption("Online Retail Dataset Preview")
    .background_gradient(cmap="Blues")
)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550000,17850.000000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390000,17850.000000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750000,17850.000000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390000,17850.000000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390000,17850.000000,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.650000,17850.000000,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.250000,17850.000000,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.850000,17850.000000,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.850000,17850.000000,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.690000,13047.000000,United Kingdom


In [41]:
print("Total Rows:", df.count())
print("Total Columns:", len(df.columns))

Total Rows: 541909
Total Columns: 8


In [42]:
# Data Cleaning
df_clean = df.dropna()
df_clean = df_clean.dropDuplicates()

print("Rows after cleaning:", df_clean.count())

Rows after cleaning: 401604


In [46]:
# Revenue Column
df_clean = df_clean.withColumn(
    "Revenue",
    col("Quantity") * col("UnitPrice")
)
# Preview
display(
    df_clean.select("Description","Quantity","UnitPrice","Revenue")
    .limit(10)
    .toPandas()
)

,Description,Quantity,UnitPrice,Revenue
0,JUMBO BAG RED RETROSPOT,2,1.95,3.90
1,AIRLINE BAG VINTAGE JET SET WHITE,1,4.25,4.25
2,HAND WARMER SCOTTY DOG DESIGN,12,2.10,25.20
3,OFFICE MUG WARMER PINK,6,2.95,17.70
4,CHILDS BREAKFAST SET DOLLY GIRL,1,9.95,9.95
5,DOORMAT I LOVE LONDON,1,7.95,7.95
6,WICKER WREATH SMALL,12,1.45,17.40
7,SET OF 20 KIDS COOKIE CUTTERS,12,2.10,25.20
8,GLASS STAR FROSTED T-LIGHT HOLDER,6,4.25,25.50
9,CLASSICAL ROSE CANDLESTAND,24,1.25,30.00


In [47]:
# Top Selling Products
top_products = df_clean.groupBy("Description") \
    .sum("Quantity") \
    .orderBy(col("sum(Quantity)").desc())

# Preview
display(
    top_products.limit(10).toPandas()
    .style.background_gradient(cmap="Greens")
    .set_caption("Top Selling Products")
)

,Description,sum(Quantity)
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53119
1,JUMBO BAG RED RETROSPOT,44963
2,ASSORTED COLOUR BIRD ORNAMENT,35215
3,WHITE HANGING HEART T-LIGHT HOLDER,34128
4,PACK OF 72 RETROSPOT CAKE CASES,33386
5,POPCORN HOLDER,30492
6,RABBIT NIGHT LIGHT,27045
7,MINI PAINT SET VINTAGE,25880
8,PACK OF 12 LONDON TISSUES,25305
9,PACK OF 60 PINK PAISLEY CAKE CASES,24129


In [48]:
# Country-wise Revenue
country_sales = df_clean.groupBy("Country") \
    .sum("Revenue") \
    .orderBy(col("sum(Revenue)").desc())
# Preview
display(
    country_sales.limit(10).toPandas()
    .style.background_gradient(cmap="Oranges")
    .set_caption("Revenue by Country")
)


,Country,sum(Revenue)
0,United Kingdom,6747156.154000
1,Netherlands,284661.540000
2,EIRE,250001.780000
3,Germany,221509.470000
4,France,196626.050000
5,Australia,137009.770000
6,Switzerland,55739.400000
7,Spain,54756.030000
8,Belgium,40910.960000
9,Sweden,36585.410000


In [49]:
# Monthly Sales Trend
# Extract month
df_clean = df_clean.withColumn(
    "Month",
    month("InvoiceDate")
)
# Aggregate
monthly_sales = df_clean.groupBy("Month") \
    .sum("Revenue") \
    .orderBy("Month")
# Convert for visualization
monthly_pd = monthly_sales.toPandas()

In [62]:
!pip install plotly
import plotly.express as px

In [85]:
# Monthly Revenue Trend
fig = px.line(
    monthly_pd,
    x="Month",
    y="sum(Revenue)",
    markers=True,
    title="Interactive Monthly Revenue Trend"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Revenue"
)

fig.show()

In [65]:
# Market Contribution --> Country Revenue
fig = px.bar(
    country_pd,
    x="sum(Revenue)",
    y="Country",
    orientation="h",
    title="Interactive Revenue by Country",
    text="sum(Revenue)"
)

fig.show()

In [68]:
# Top Products
fig = px.bar(
    top_products_pd,
    x="sum(Quantity)",
    y="Description",
    orientation="h",
    title="Top Selling Products"
)

fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [73]:
# User interactive country wise transactions
from ipywidgets import interact
from IPython.display import display

@interact(country=country_pd["Country"].tolist())
def show_country(country):

    # Filter data
    filtered_df = (
        df_clean
        .filter(col("Country") == country)
        .select("Description", "Quantity", "UnitPrice", "Revenue")
        .limit(10)
        .toPandas()
    )

    # Professional styled table
    styled_table = (
        filtered_df.style
        .set_caption(f"Top Transactions — {country}")
        .set_table_styles([
            {"selector": "th",
             "props": [("background-color", "#2c3e50"),
                       ("color", "white"),
                       ("text-align", "center")]},
            {"selector": "td",
             "props": [("text-align", "center")]},
            {"selector": "caption",
             "props": [("font-size", "16px"),
                       ("font-weight", "bold")]}
        ])
        .set_properties(**{
            "background-color": "white",
            "border": "1px solid #ddd"
        })
    )

    display(styled_table)

interactive(children=(Dropdown(description='country', options=('United Kingdom', 'Netherlands', 'EIRE', 'Germa…

In [84]:
# SCALABILITY DEMONSTRATION USING PYSPARK

import pandas as pd
import time
from pyspark.sql.functions import spark_partition_id

print("\n--- Scalability Demonstration Started ---")

# Check Original Number of Partitions
print("\nOriginal Partitions:",
      df.rdd.getNumPartitions())

# Increase Partitions (Simulating Scalability)
df_scaled = df.repartition(8)

print("Partitions After Scaling:",
      df_scaled.rdd.getNumPartitions())

# Pandas Execution Timing (Single Machine Processing)
start = time.time()

pandas_df = pd.read_csv("/content/online_retail.csv")
pandas_result = (
    pandas_df.groupby("Country")["Quantity"]
    .sum()
)

end = time.time()

print("\nPandas Total Execution Time:",
      f"{end - start:.3f} seconds")


# PySpark Execution Timing (Original Dataset)

start = time.time()

df.groupBy("Country").sum("Quantity").show()

end = time.time()

print("PySpark Execution Time (Original):",
      f"{end - start:.3f} seconds")

# PySpark Execution Timing AFTER SCALING

start = time.time()

df_scaled.groupBy("Country").sum("Quantity").show()

end = time.time()

print("PySpark Execution Time (Scaled):",
      f"{end - start:.3f} seconds")
# Show Partition Distribution (Proof of Parallelism)

print("\nPartition Distribution:")
df_scaled.groupBy(spark_partition_id()).count().show()



# Spark Execution Plan (Distributed Processing Proof)

print("\nExecution Plan:")
df_clean.groupBy("Country").sum("Revenue").explain(True)

print("\n--- Scalability Demonstration Completed ---")


--- Scalability Demonstration Started ---

Original Partitions: 2
Partitions After Scaling: 8

Pandas Total Execution Time: 1.086 seconds
+------------------+-------------+
|           Country|sum(Quantity)|
+------------------+-------------+
|            Sweden|        35637|
|         Singapore|         5234|
|           Germany|       117448|
|            France|       110480|
|            Greece|         1556|
|European Community|          497|
|           Belgium|        23152|
|           Finland|        10666|
|             Malta|          944|
|       Unspecified|         3300|
|             Italy|         7999|
|              EIRE|       142637|
|         Lithuania|          652|
|            Norway|        19247|
|             Spain|        26824|
|           Denmark|         8188|
|         Hong Kong|         4769|
|           Iceland|         2458|
|            Israel|         4353|
|   Channel Islands|         9479|
+------------------+-------------+
only showing top 20 r

**Analytics:**

1.Sales Performance Analysis

2.Revenue Trends

3.Interactive Exploration

4.Big Data Processing


**Outcome:**

This work demonstrated big data analysis using PySpark to clean, transform, and analyze retail transaction data for business insights.
Scalability was shown through distributed processing and a performance comparison with Pandas, highlighting differences between single-machine and distributed frameworks.
While Pandas performed faster on small data, PySpark proved more suitable for large-scale analytics requiring scalability and efficient processing.
